In [1]:
from pyspark import SparkContext, SparkConf, Row
from pyspark.sql import SparkSession

from operator import add

In [2]:
# Spark Constants
APP_NAME = 'assignment1'
MASTER = 'local[*]'

# Input Constants
INPUT_METADATA_FILE = 'tracks.csv'
INPUT_FEATURES_FILE = 'features.csv'

In [3]:
conf = SparkConf().setAppName(APP_NAME).setMaster(MASTER)
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession.builder.appName(APP_NAME).master(MASTER).getOrCreate()

23/04/06 21:09:24 WARN Utils: Your hostname, pedro-duarte resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp2s0)
23/04/06 21:09:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/06 21:09:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [31]:
data = spark.read.csv(INPUT_METADATA_FILE, quote='"', escape='"', multiLine=True).rdd
headerRow = ['_'.join(header) if idx > 0 else 'track_id' for idx, header in enumerate(zip(*[list(line.asDict().values()) for line in data.take(2)]))]

headerRows = data.take(3)
data = data.filter(lambda row: row not in headerRows)

firstRow = data.first()

data = data \
  .map(lambda row: list(row.asDict().values())) \
  .map(lambda row: [v if v != None else '' for v in row]) \
  .map(lambda row: {k: v for k, v in zip(headerRow, row)})

ds = spark.read.json(data)
ds.schema

StructType([StructField('_corrupt_record', StringType(), True), StructField('album_comments', StringType(), True), StructField('album_date_created', StringType(), True), StructField('album_date_released', StringType(), True), StructField('album_engineer', StringType(), True), StructField('album_favorites', StringType(), True), StructField('album_id', StringType(), True), StructField('album_information', StringType(), True), StructField('album_listens', StringType(), True), StructField('album_producer', StringType(), True), StructField('album_tags', StringType(), True), StructField('album_title', StringType(), True), StructField('album_tracks', StringType(), True), StructField('album_type', StringType(), True), StructField('artist_active_year_begin', StringType(), True), StructField('artist_active_year_end', StringType(), True), StructField('artist_associated_labels', StringType(), True), StructField('artist_bio', StringType(), True), StructField('artist_comments', StringType(), True), 

In [20]:
ds.rdd.count()

106574

In [38]:
ds.rdd.map(lambda v: v['set_subset']).filter(lambda v: v not in ('small', 'medium', 'large'))

[None]

In [33]:
data = ds.rdd.filter(lambda v: v['set_subset'] == 'small')
data.count() # 8000 lines

4483